# Demo for running the code for the manuscript "Title of the manusc"
This demo is intended to be used as an example of the main steps including:
- training: ...
- inference: ...
- evaluation:....

## Setup the environment

In [ ]:
import ultralytics
from ultralytics import YOLO

import cv2
import numpy as np
import os
import pandas as pd
import shutil
import random

import matplotlib.pyplot as plt

# import function to predict on raw images and estimate percentage cover per species 
from functions import predict_cover, validate_cover


## Training
bla bla bla

In [1]:

###################################
### Train frame detection model ###
###################################

# path to training data for frame detection model 
path_to_data_frame="path_to/project_location" # path to the root where you have the yaml and the images and labels subfolder


### Structure of the folder for training the model
-root
      -images
              -img_1.jpeg
              -img_2.jpeg
              -...
      -labels
              -img_1.txt
              -img_2.txt
              -...
      -dataset.yaml
              -names:
                0: class1
                1: class2
                ...
              -path: /content/data/project_title
              -train: train.txt
              -val: val.txt
      -train.txt
              -./images/img_1.jpg
      -val.txt
              -./images/img_2.jpg


In [2]:

# Set YOLO parameters
IM_SIZE = 640 # Image size for training
N_EPOCHS = 50 # Number of epochs for training
BATCH_SIZE = 4 # Batch size for training
MODEL = 'yolov8n.pt' # Choose pretrained YOLO model version and size

# Train frame detection model
!yolo task=detect mode=train model=$MODEL data="{path_to_data_frame}/dataset.yaml", epochs=$N_EPOCHS, imgsz=$IM_SIZE, batch=$BATCH_SIZE, project=path_to_data_frame, name="species_detector_imgsz"+str(IM_SIZE)+"_modelSizeN"

# Construct the path to the trained model's best weights
trained_model_path_frame = f'{path_to_data_frame}/species_detector_imgsz{IM_SIZE}_modelSizeN/weights/best.pt'

########################################
### Train species segmentation model ###
########################################

# path to training data for species segmentation model 
path_to_data_species="path_to/project_location" # path to the root where you have the yaml and the images and labels subfolder

# Train species segmentation model
!yolo task=segment mode=train model=$MODEL data="{path_to_data_species}/dataset.yaml", epochs=$N_EPOCHS, imgsz=$IM_SIZE, batch=$BATCH_SIZE, project=path_to_data_species, name="species_detector_imgsz"+str(IM_SIZE)+"_modelSizeN"

# Construct the path to the trained model's best weights
trained_model_path_species = f'{path_to_data_species}/species_detector_imgsz{IM_SIZE}_modelSizeN/weights/best.pt'

Traceback (most recent call last):
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\stpu\Anaconda3\envs\deepField\Scripts\yolo.exe\__main__.py", line 4, in <module>
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\site-packages\ultralytics\__init__.py", line 10, in <module>
    from ultralytics.data.explorer.explorer import Explorer
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\site-packages\ultralytics\data\__init__.py", line 3, in <module>
    from .base import BaseDataset
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\site-packages\ultralytics\data\base.py", line 15, in <module>
    from torch.utils.data import Dataset
  File "C:\Users\stpu\Anaconda3\envs\deepField\lib\site-packages\torch\__init__.py", line 148, in <module>
    raise err
OSError: [WinErr

## Inference
bla bla bla

In [ ]:
# path to raw images 
path_to_data_images = "path_to/images" # path to the folder containing the photos taken in the field

# predict on raw images 
predict_cover(path_to_data_images, path_to_model_frame=trained_model_path_frame, name_frame_class="frame", # parameter for cropping to frame
            path_to_model_species=trained_model_path_species, conf_treshold=0.10,                     # parameter for species segmentation
            number_of_classes=2)                                                                    # parameter for calculating percentage cover


## Evaluation
bla bla bla

In [ ]:
validate(field_data, # (str) path to field estimations of coverage by fieldworkers
         predictions, # (str) path to predicted coverage
         image_ID_column, # (str) column which exists in both datasets (Image ID)
         classes_to_include, # (lst) column names of classes to include in the validation (must be the same in field and prediction dataset)
         names_of_classes, # (lst) names to be used in the legend of the scatterplot
         colors) # (lst) colours to be used in the scatterplot